In [1]:
###如果 center=False（默认设置），则 closed 默认为 'right'。这意味着窗口是左闭右开的，即窗口包括左边界上的点，但不包括右边界上的点
###如果 center=True，则 closed 默认为 'both'。这意味着窗口是两边都闭的，即窗口包括左边界和右边界上的点。

import os
import sys
root_directory = os.getcwd()[0:3]
sys.path.append(root_directory+'SCI\\SCI9\\2_DataAnalysis\\04_code\\1_04_Soybean')
sys.path.append(r'C:\ProgramData\anaconda3\Lib\site-packages') 
sys.path.append(r'C:\Users\DELL\.conda\envs\myenv\Lib\site-packages') 
sys.path.append(r'C:\Users\DELL\.conda\envs\rasterio_env\Lib\site-packages') 
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
# 添加新的环境变量
import nbimporter 
from functions import spei,extreme_temperature,aggre_8days,aggre_8days_modis

T_upper=34
T_lower=0

root_directory = os.getcwd()[0:3]
crop = '02_wheat';
countryID ='06_India'
country =countryID[3:]
inputpath_base = root_directory + '\\SCI\\SCI9_1\\02_data\\'+crop+'\\'+countryID
inpath_dates_other = root_directory + '\\SCI\\SCI9_1\\02_data\\'+crop+'\\'+countryID+'\\'+'01_data'+'\\'+'07_Information'
other_infornamtion = pd.read_csv(os.path.join(inpath_dates_other,'information.txt'), sep=' ', header=None)
startyear,endyear,shp_name = other_infornamtion.iloc[0,0],other_infornamtion.iloc[0,1],other_infornamtion.iloc[0,2]

startyear = 2001 # 模型开始的日期
yieldyear = endyear # 产量记录开始日期
years = range(startyear,endyear+1)
years_str = [str(year) for year in years]

VIs =  [ '_KNDVI' ,'_EVI','_NDVI']
Cilmate = ['_Pre' ,'_Tmin' ,'_Solar','_Tmean','_Tmax']
Climate_Exogenous  = ['_CDD' ,'_HDD' ,'_GDD','_VPD','_wind_speed','_SPEI'] #'_VPD','_wind_speed',
soil_feature = [ 'SAND','AWC', 'SILT','ORG_CARBON',  'TOTAL_N', 'PH_WATER',  'CEC_SOIL', 'CLAY']
loc_feature = ['elevation', 'lat', 'lon']
Year_feature = ['year'];union_feature = ['idJoin'];
dynamic_features = [ '_KNDVI' ,'_EVI','_NDVI','_Pre' ,'_Tmin' ,'_Solar','_Tmean','_VPD', '_wind_speed' ,'_Tmax']

C:\Users\DELL\.conda\envs\myenv\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
regions =  ['I']
for region in regions:
    pre_filename = 'WinterWheat_'+country+region+'_'
    dataall = pd.DataFrame()
    for year in years:
        data_new = pd.read_csv(os.path.join(inputpath_base, '01_data','04_GEEdownloadData','01_DailyData', pre_filename + str(year) + '.csv')).iloc[:, 1:]
        Tmin_columns = [col for col in data_new.columns if '_Tmin' in col]
        Tmin = data_new[Tmin_columns].values;
        Tmean_columns = [col for col in data_new.columns if '_Tmean' in col]
        Tmean = data_new[Tmean_columns].values;
        Tmax_columns = [col for col in data_new.columns if '_Tmax' in col]
        Tmax = data_new[Tmax_columns].values;
        Pre_columns = [col for col in data_new.columns if '_Pre' in col]
        Pre = data_new[Pre_columns].values;
        days = Pre.shape[1]
        dates = pd.date_range(start=str(year)+'-01-01', periods=days, freq='D')
        data_new['year'] = year
        # 计算极端气象指标
        spei_df = spei(dates, Pre, Tmean)
        CDD_df,HDD_df,GDD_df = extreme_temperature(dates,Tmax,Tmin,T_upper,T_lower) # 有问题
        data_new1 = aggre_8days(dynamic_features,dates,data_new)
        data_new1 = pd.concat([CDD_df, HDD_df,GDD_df,spei_df,data_new1,data_new[soil_feature+loc_feature+Year_feature+union_feature]], axis=1)
        dataall = pd.concat([dataall, data_new1])
    
    outputpath_base = os.path.join(inputpath_base, '01_data','05_buildmodel','01_weekdata')
    os.makedirs(outputpath_base, exist_ok=True)
    dataall.to_csv(os.path.join(outputpath_base,region+'_allweekdata.csv'),index=False)#

In [3]:
#############################02_modis8aggre################################
dynamic_features_8sggre =['_modis_Gpp','_modis_Fpar','_modis_LAI','_PML_Gpp']

for region in regions:
    pre_filename2 = 'WinterWheat'+ '_8agg' + country+region + '_'    
    dataall = pd.DataFrame()
    for year in years:
        data_new = pd.read_csv(os.path.join(inputpath_base, '01_data','04_GEEdownloadData','01_WeekotherData',pre_filename2 + str(year) + '.csv')).iloc[:, 1:]
        dates = pd.date_range(start=str(year)+'-01-01', end=str(year)+'-12-31', freq='8D')
        data_new1 = aggre_8days_modis(dynamic_features_8sggre,dates,data_new)
        data_new1['year'] = year
        data_new1['idJoin'] = data_new['idJoin']
        dataall = pd.concat([dataall, data_new1])
    #dataall = dataall.dropna(how='all',axis=1)
    for col in dynamic_features_8sggre:
        dataname = [f'Week{week}{col}' for week in range(1, 47)]
        dataall[dataname] = dataall[dataname].apply(lambda row: row.fillna(row.rolling(4, min_periods=1).mean()), axis=1)
    dataall.fillna(0, inplace=True)
    outputpath_base = os.path.join(inputpath_base, '01_data','05_buildmodel','01_weekdata',region+'_allweekdata_8modis.csv')
    dataall.to_csv(outputpath_base,index=False)

In [4]:
################03_join_yieldsandalldatasRegion########################
for region in regions:
    data_new1 = pd.read_csv(os.path.join(inputpath_base, '01_data','05_buildmodel','01_weekdata',region+'_allweekdata.csv'))
    data_new2 = pd.read_csv(os.path.join(inputpath_base, '01_data','05_buildmodel','01_weekdata',region+'_allweekdata_8modis.csv'))
    inputpath_base_Yield =  os.path.join(inputpath_base,'01_data', '01_Yield',country+'_winter.csv')
    long_df = pd.read_csv(inputpath_base_Yield)
    

    # 最近五年的均值,可以捕获到
    long_df = pd.read_csv(inputpath_base_Yield)
    long_df_mean = long_df.copy()
    data = long_df_mean.loc[:,str(yieldyear):].rolling(window=5, center=False, min_periods=2, closed='right',axis=1).mean() #前五年的数据，不包括当前的点
    long_df_mean.loc[:,str(yieldyear):] = data # str(yieldyear)是空值，所以要插补下
    long_df_mean.loc[:,str(yieldyear)] = long_df_mean.loc[:,str(yieldyear+1)].values
    
    # 前一年的产量作为潜在产量
    long_df_pre = long_df.copy()
    data = long_df_pre.loc[:,str(yieldyear):str(endyear)].values[:,:-1]
    long_df_pre.loc[:,str(yieldyear+1):] = data
    long_df_pre.loc[:,str(yieldyear)] = long_df.loc[:,str(yieldyear+1)] # 最开始哪一年空值，
    
    long_df = pd.melt(long_df,id_vars=['idJoin'], var_name='year',  value_name='Yield',value_vars=['idJoin']+years_str)
    if long_df['year'].dtype == 'object':  
        long_df['year'] = long_df['year'].astype(int)  
        
   # long_df = long_df.drop(['Country'],axis=1)
    long_df_mean = pd.melt(long_df_mean,id_vars=['idJoin'], var_name='year',  value_name='Mean_Yield',value_vars=['idJoin']+years_str)
    if long_df_mean['year'].dtype == 'object':  
        long_df_mean['year'] = long_df_mean['year'].astype(int)     
        
    long_df_pre = pd.melt(long_df_pre,id_vars=['idJoin'], var_name='year',  value_name='Previous_Yield',value_vars=['idJoin']+years_str)
    if long_df_pre['year'].dtype == 'object':  
        long_df_pre['year'] = long_df_pre['year'].astype(int) 


    
    data_yield_ori_Visall = data_new1.merge(long_df, how='left', on=['idJoin', 'year']).dropna(axis=1, how='all')
    
    data_yield_ori_Visall = long_df_mean.merge(data_yield_ori_Visall, how='inner', on=['idJoin', 'year']).dropna(axis=1, how='all') # 链接均值
    data_yield_ori_Visall = data_new2.merge(data_yield_ori_Visall, how='inner', on=['idJoin', 'year']).dropna(axis=1, how='all')
    data_yield_ori_Visall = long_df_pre.merge(data_yield_ori_Visall, how='inner', on=['idJoin', 'year']).dropna(axis=1, how='all')
    outputpath_base = os.path.join(inputpath_base, '01_data','05_buildmodel','01_weekdata',region+'_allweekYielddata.csv')
    data_yield_ori_Visall.to_csv(outputpath_base,index=False)

In [11]:
 # 这里之前的算法会用到前一年的产量，所以要比之前多一年
# 有前几年的产量就算前几年的产量均值，及其前一年
# 如果没有的话，就取后面的所有算作均值，后一年的产量作为均值



In [8]:

# year+1
# merge
# 重新赋值2003


idJoin  year        Yield     PreYield
0         CHN_0  2003  2668.987300   989.000000
1         CHN_1  2003  2668.987300  1665.000000
2      CHN_1000  2003  2668.987300  1665.000000
3      CHN_1001  2003  2668.987300  1665.000000
4      CHN_1003  2003  2668.987300  1665.000000
...         ...   ...          ...          ...
24843   CHN_990  2018  2325.343986  2232.668566
24844   CHN_995  2018  2295.907660  2190.875233
24845   CHN_996  2018  1785.294118  1441.613588
24846   CHN_998  2018  1785.294118  1441.613588
24847   CHN_999  2018  1785.294118  1441.613588

[24848 rows x 4 columns]